<a href="https://colab.research.google.com/github/AshOne91/PLAYDATA-COLAB/blob/main/20250519_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 의료 도메인
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
model = AutoModelForQuestionAnswering.from_pretrained("beomi/KcELECTRA-base")
qa = pipeline('question-answering', model=model, tokenizer=tokenizer)
result = qa({
    "context" : '당뇨병은 인슐린의 부족이나 작용의 이상으로 인해 혈당이 상승하는 질환이다'
    ,"question" : "당뇨병은 왜 발생하나요?"
})
result

Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


{'score': 0.0041921790689229965, 'start': 0, 'end': 9, 'answer': '당뇨병은 인슐린의'}

In [ ]:
train_data = [
  {
    "question": "당뇨병의 주요 증상은 무엇인가요?",
    "context": "당뇨병은 인슐린 기능 저하로 인해 혈당이 비정상적으로 높아지는 질환입니다.",
    "answer": "대표적인 증상으로는 잦은 소변, 극심한 갈증, 피로감, 체중 감소 등이 있습니다."
  },
  {
    "question": "고혈압은 왜 위험한가요?",
    "context": "고혈압은 혈관에 지속적으로 높은 압력이 가해지는 상태로, 다양한 합병증을 유발할 수 있습니다.",
    "answer": "심근경색, 뇌졸중, 신부전 등 심각한 합병증의 위험이 높아집니다."
  },
  {
    "question": "천식 환자가 피해야 할 환경은?",
    "context": "천식은 기관지의 만성 염증성 질환으로 특정 자극에 민감하게 반응합니다.",
    "answer": "먼지, 꽃가루, 동물의 털, 찬 공기, 강한 냄새가 있는 환경을 피해야 합니다."
  },
  {
    "question": "간염 A형은 어떻게 전염되나요?",
    "context": "A형 간염은 바이러스 감염에 의해 발생하는 급성 간 질환입니다.",
    "answer": "주로 오염된 음식이나 물을 통해 경구 감염되며, 위생 상태가 불량한 지역에서 전염이 잘 됩니다."
  },
  {
    "question": "빈혈의 일반적인 원인은?",
    "context": "빈혈은 혈액 내 적혈구 수나 헤모글로빈 농도가 정상보다 낮을 때 발생합니다.",
    "answer": "철분 결핍, 만성 출혈, 비타민 B12 부족 등이 일반적인 원인입니다."
  },
  {
    "question": "골다공증의 예방 방법은?",
    "context": "골다공증은 뼈의 밀도와 강도가 감소하여 쉽게 골절되는 질환입니다.",
    "answer": "칼슘과 비타민 D 섭취, 규칙적인 운동, 금연 및 음주 제한이 예방에 도움이 됩니다."
  },
  {
    "question": "백신은 어떤 방식으로 면역을 유도하나요?",
    "context": "백신은 면역 체계가 특정 병원체를 인식하고 기억하도록 도와줍니다.",
    "answer": "백신은 병원체의 일부나 비활성화된 형태를 주입하여 항체 생성을 유도합니다."
  },
  {
    "question": "심근경색과 협심증의 차이점은?",
    "context": "둘 다 관상동맥과 관련된 심장 질환이지만, 병리학적으로 차이가 있습니다.",
    "answer": "협심증은 일시적인 혈류 감소로 인한 통증이며, 심근경색은 혈류가 완전히 차단되어 심장 조직이 괴사하는 상태입니다."
  },
  {
    "question": "대장내시경 검사는 언제 받아야 하나요?",
    "context": "대장내시경은 대장암 및 대장질환 조기 발견을 위한 검사입니다.",
    "answer": "일반적으로 50세 이상에서 5~10년 간격으로 검사를 권장합니다. 가족력이 있으면 더 이르게 시작할 수 있습니다."
  },
  {
    "question": "독감과 감기의 차이는?",
    "context": "두 질환 모두 바이러스 감염이지만 증상과 심각도에 차이가 있습니다.",
    "answer": "독감은 고열과 근육통이 동반되고 갑작스럽게 시작되며, 감기는 점진적인 증상과 비교적 경미한 증상을 보입니다."
  }
]

In [ ]:
!pip install datasets

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# 모델 선택(gpt, bert... 등등)
# 모델명은 skt/kogpt2-base-v2   -transformer   input_ids masked. inputs
# 가이드를 참고해서 model tokenizer
# LoRA 구성   LoRA config
# model = get_peft_model(model, lora_config)
# 학습용 데이터
'''
def preprocess(example):
    prompt = f"질문: {example['question']}\n문맥: {example['context']}\n답변:"
    inputs = tokenizer(prompt, truncation=True, padding='max_length', max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['answer'], truncation=True, padding='max_length', max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs
dataset = Dataset.from_list(data)
tokenized_dataset = dataset.map(preprocess, batched=True)
'''
# 학습 설정 trainer 구성
# TraningArgments
# Traner
# train

In [ ]:
# 약간의 대용량 DATA + QA 시스템구현
# 문맥기반 추론형(skt/koBert) QA or  정답 추출형 QA (electra)

In [ ]:
# 정답추출형 QA : 정답이 되는 Text를 직접 추출

```
사전학습모델 로드 -> LoRA설정
-> 모델에 LoRA 적용->데이터셋 전처리
->Dataset 객체구성->Trainer 학습

```

한글 QA 데이터셋 로드 및 QA 구조로 변경

In [ ]:
!pip install emoji
!pip install soynlp

In [ ]:
from transformers import AutoTokenizer, AutoModel
model_name = 'beomi/KcELECTRA-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

import re
import emoji
from soynlp.normalizer import repeat_normalize

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: model_name is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from datasets import load_dataset
import json
# dataset = load_dataset('json',data_files='K.json',cache_dir=None)
with open('K.json','r',encoding='utf-8') as f:
    dataset = json.load(f)

dataset
# def get_qa(datas = dataset['train'][0]['paragraphs'][0]):
#     for row in datas['qas']:
#         # prompt = f"질문: {row['question']}\n문맥: {datas['context']}\n답변:{row['answers'][0]['text']}"
#         return {
#             '질문' : row['question'],
#             '문맥' : datas['context'],
#             '답변' : row['answers'][0]['text']
#         }
# temp_datasets = []
# for rows in dataset['train']:
#     for row in rows['paragraphs']:
#         temp_datasets.append(get_qa(row))

In [ ]:
import os
os.path.exists('/content/train.json')

True